In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize
AI_MODEL = initialize()

import asyncio
from typing import List, Dict
from pprint import pprint, pformat

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']
Using AI model: openai:gpt-4o


In [3]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of task. '
        'For example: "formal", "conversational", "hybrid", ...'))
    description: str = Field(..., description='Clear description for executing this task.')


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the task and which variations would be valuable. '
        'Focus on how each approach serves different aspects of the task.'
    ))
    tasks: List[Task] = Field(..., description="List of tasks")


async def orchestrate(task: str) -> Dict:
    """Process task by breaking it down and running subtasks in parallel."""

    orchestrator_response = await Agent(
        AI_MODEL,
        system_prompt='Analyze this task and break it down into 2-3 distinct approaches.',
        result_type=OrchestratorResponse,
    ).run(task)
    
    # Parse orchestrator response
    analysis = orchestrator_response.data.analysis
    tasks = orchestrator_response.data.tasks
    
    print("\n=== ORCHESTRATOR OUTPUT ===")
    print(f"\nANALYSIS:\n{analysis}")
    print(f"\nTASKS:")
    for task in tasks:
        pprint(task.model_dump())
    
    # Step 2: Process all the tasks in parallel and collect results
    worker_agent = Agent(AI_MODEL, system_prompt='Generate content based on the task specification.')
    worker_responses = await asyncio.gather(*[
        worker_agent.run(pformat({'original_task': task} | task_info.model_dump()))
        for task_info in tasks
    ])
    worker_results = [
        {
            'type': task.type,
            'description': task.description,
            'result': response.data,
        }
        for task, response in zip(tasks, worker_responses)
    ]

    for worker_result in worker_results:
        print(f"\n=== WORKER RESULT ({worker_result['type']}) ===\n{worker_result['result']}\n")
    
    return {
        "analysis": analysis,
        "worker_results": worker_results,
    }

In [4]:
results = await orchestrate('Write a product description for a new eco-friendly water bottle.')


=== ORCHESTRATOR OUTPUT ===

ANALYSIS:
Writing a product description for an eco-friendly water bottle can be approached in various ways depending on the target audience and the emphasis needed. One can choose a formal tone to appeal to businesses and environmentally conscious professionals, a conversational tone to engage a wide range of consumers casually, or a hybrid approach that combines both tones for a broader appeal. Each approach highlights different aspects of the product, such as environmental benefits, practical features, or lifestyle integration.

TASKS:
{'description': 'Craft a formal product description that highlights the '
                'eco-friendly features of the water bottle, such as '
                'biodegradable materials, sustainability impact, and any '
                'certifications it may have. Tailor the language to appeal to '
                'environmentally conscious consumers, corporations, or buyers '
                'who value formal communication